In [ ]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 50

// CUDA kernel to perform substring check
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        // If we matched the full length of the search string
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Updated kernel using flattened arrays instead of a struct
__global__ void searchPhonebook(char* d_names, char* d_numbers, int num_contacts, char* search_name, int name_length) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        // Calculate the starting pointer for the current contact's name and number
        char* current_name = d_names + (idx * MAX_STR_LEN);
        char* current_number = d_numbers + (idx * MAX_STR_LEN);

        if(check(current_name, search_name, name_length)) {
            printf("%s %s\n", current_name, current_number);
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "/content/sample_data/phonebook1.txt";

    // 1. Read all lines into a vector
    vector<string> raw_lines;
    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(!line.empty()) raw_lines.push_back(line);
    }
    file.close();

    int num_contacts = raw_lines.size();

    // 2. Prepare flattened host memory (Structure of Arrays approach)
    // We use a flat array where every 50 bytes represents one name/number
    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN * sizeof(char));
    char* h_numbers = (char*)malloc(num_contacts * MAX_STR_LEN * sizeof(char));

    for(int i = 0; i < num_contacts; i++) {
        string current_line = raw_lines[i];
        int pos = current_line.find(",");

        // Extract Name and Number from CSV format: "Name","Number"
        string name = current_line.substr(1, pos - 2);
        string number = current_line.substr(pos + 2, current_line.size() - pos - 3);

        // Copy strings into the flat buffer at specific offsets
        strncpy(h_names + (i * MAX_STR_LEN), name.c_str(), MAX_STR_LEN - 1);
        strncpy(h_numbers + (i * MAX_STR_LEN), number.c_str(), MAX_STR_LEN - 1);

        // Ensure null termination
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
        h_numbers[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    // 3. Allocate and Copy to Device
    char *d_names, *d_numbers, *d_search_name;
    int name_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_numbers, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_search_name, name_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_numbers, h_numbers, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), name_len, cudaMemcpyHostToDevice);

    // 4. Launch Kernel
    // We handle the data in chunks based on num_threads as per your original logic
    for(int i = 0; i < num_contacts; i += num_threads) {
        int thread_count = min(num_contacts - i, num_threads);

        // Pass the pointers shifted by the current offset 'i'
        searchPhonebook<<<1, thread_count>>>(
            d_names + (i * MAX_STR_LEN),
            d_numbers + (i * MAX_STR_LEN),
            thread_count,
            d_search_name,
            name_len
        );
        cudaDeviceSynchronize();
    }

    // 5. Cleanup
    free(h_names);
    free(h_numbers);
    cudaFree(d_names);
    cudaFree(d_numbers);
    cudaFree(d_search_name);

    return 0;
}

//!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook
//!pip install pycuda
//!time ./search_phonebook SHAKIRA 100 > output1.txt

Writing search_phonebook.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [ ]:
!time ./search_phonebook LA 100 > output1.txt


real	0m0.133s
user	0m0.015s
sys	0m0.111s


**Sorting** Ascensing orders

In [ ]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 50

// Helper struct for sorting on the CPU
struct ResultContact {
    string name;
    string number;

    // Logic for ascending sort by name
    bool operator<(const ResultContact& other) const {
        return name < other.name;
    }
};

// Substring check logic
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Kernel writes a '1' to the results array if a match is found
__global__ void searchPhonebook(char* d_names, int num_contacts, char* search_name, int name_length, int* d_results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        char* current_name = d_names + (idx * MAX_STR_LEN);
        if(check(current_name, search_name, name_length)) {
            d_results[idx] = 1; // Mark as found
        } else {
            d_results[idx] = 0; // Not found
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "/content/sample_data/phonebook1.txt";

    // Vectors to hold raw data for processing
    vector<string> host_names_vec;
    vector<string> host_numbers_vec;

    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(line.empty()) continue;
        int pos = line.find(",");
        string name = line.substr(1, pos - 2);
        string number = line.substr(pos + 2, line.size() - pos - 3);
        host_names_vec.push_back(name);
        host_numbers_vec.push_back(number);
    }
    file.close();

    int num_contacts = host_names_vec.size();

    // Flatten data for CUDA
    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN);
    int* h_results = (int*)malloc(num_contacts * sizeof(int));

    for(int i = 0; i < num_contacts; i++) {
        strncpy(h_names + (i * MAX_STR_LEN), host_names_vec[i].c_str(), MAX_STR_LEN - 1);
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    // Allocate Device Memory
    char *d_names, *d_search_name;
    int *d_results;
    int search_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_results, num_contacts * sizeof(int));
    cudaMalloc((void**)&d_search_name, search_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), search_len, cudaMemcpyHostToDevice);

    // Launch Kernel (Using grid/block logic based on num_threads)
    int blocks = (num_contacts + num_threads - 1) / num_threads;
    searchPhonebook<<<blocks, num_threads>>>(d_names, num_contacts, d_search_name, search_len, d_results);
    cudaDeviceSynchronize();

    // Copy results back to host
    cudaMemcpy(h_results, d_results, num_contacts * sizeof(int), cudaMemcpyDeviceToHost);

    // Filter matched contacts into a vector for sorting
    vector<ResultContact> matched_contacts;
    for(int i = 0; i < num_contacts; i++) {
        if(h_results[i] == 1) {
            matched_contacts.push_back({host_names_vec[i], host_numbers_vec[i]});
        }
    }

    // Sort the results in ascending order
    sort(matched_contacts.begin(), matched_contacts.end());

    // Print final sorted output
    cout << "Search Results (Ascending Order):" << endl;
    for(const auto& contact : matched_contacts) {
        cout << contact.name << " " << contact.number << endl;
    }

    // Cleanup
    free(h_names);
    free(h_results);
    cudaFree(d_names);
    cudaFree(d_results);
    cudaFree(d_search_name);

    return 0;
}

Overwriting search_phonebook.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [ ]:
!time ./search_phonebook LA 100 > output1.txt


real	0m0.129s
user	0m0.019s
sys	0m0.106s


Desending Order

In [ ]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 50

// Helper struct for sorting on the CPU
struct ResultContact {
    string name;
    string number;

    // CHANGE HERE: Changed < to > for Descending Order
    bool operator<(const ResultContact& other) const {
        return name > other.name;
    }
};

// Substring check logic (GPU)
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Kernel writes a '1' to the results array if a match is found
__global__ void searchPhonebook(char* d_names, int num_contacts, char* search_name, int name_length, int* d_results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        char* current_name = d_names + (idx * MAX_STR_LEN);
        if(check(current_name, search_name, name_length)) {
            d_results[idx] = 1;
        } else {
            d_results[idx] = 0;
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "/content/sample_data/phonebook1.txt";

    vector<string> host_names_vec;
    vector<string> host_numbers_vec;

    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(line.empty()) continue;
        int pos = line.find(",");
        string name = line.substr(1, pos - 2);
        string number = line.substr(pos + 2, line.size() - pos - 3);
        host_names_vec.push_back(name);
        host_numbers_vec.push_back(number);
    }
    file.close();

    int num_contacts = host_names_vec.size();

    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN);
    int* h_results = (int*)malloc(num_contacts * sizeof(int));

    for(int i = 0; i < num_contacts; i++) {
        strncpy(h_names + (i * MAX_STR_LEN), host_names_vec[i].c_str(), MAX_STR_LEN - 1);
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    char *d_names, *d_search_name;
    int *d_results;
    int search_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_results, num_contacts * sizeof(int));
    cudaMalloc((void**)&d_search_name, search_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), search_len, cudaMemcpyHostToDevice);

    int blocks = (num_contacts + num_threads - 1) / num_threads;
    searchPhonebook<<<blocks, num_threads>>>(d_names, num_contacts, d_search_name, search_len, d_results);
    cudaDeviceSynchronize();

    cudaMemcpy(h_results, d_results, num_contacts * sizeof(int), cudaMemcpyDeviceToHost);

    vector<ResultContact> matched_contacts;
    for(int i = 0; i < num_contacts; i++) {
        if(h_results[i] == 1) {
            matched_contacts.push_back({host_names_vec[i], host_numbers_vec[i]});
        }
    }

    // Sorts using the modified operator (Descending)
    sort(matched_contacts.begin(), matched_contacts.end());

    cout << "Search Results (Descending Order):" << endl;
    for(const auto& contact : matched_contacts) {
        cout << contact.name << " " << contact.number << endl;
    }

    free(h_names);
    free(h_results);
    cudaFree(d_names);
    cudaFree(d_results);
    cudaFree(d_search_name);

    return 0;
}

Overwriting search_phonebook.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [ ]:
!time ./search_phonebook LA 100 > output1.txt


real	0m0.130s
user	0m0.011s
sys	0m0.111s
